In [57]:
import pandas as pd 
import numpy as np

In [58]:
from sklearn.datasets import load_iris
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [59]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['terget'] = iris.target
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),terget
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [60]:
from sklearn.model_selection import train_test_split
x = iris.data
y = iris.target
x_tarin, x_test, y_tarin, y_test = train_test_split(x,y, test_size=0.2)

the value might vary on test_size, or paramitter

In [61]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',C=30, gamma='auto')
model.fit(x_tarin, y_tarin)
model.score(x_test, y_test)

1.0

crossval score 

In [62]:
from sklearn.model_selection import cross_val_score

In [63]:
cross_val_score(SVC(kernel='rbf',C=10, gamma='auto'), iris.data, iris.target, cv= 5)
# cv = cross validation or ittration 

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [64]:
cross_val_score(SVC(kernel='rbf',C=20, gamma='auto'), iris.data, iris.target, cv= 5)
# cv = cross validation or ittration 

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

whole this process in loop

In [65]:
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [66]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
},
 cv=5, 
 return_train_score=False)
 
clf.fit(iris.data, iris.target)
#clf.cv_results_

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [67]:
df = pd.DataFrame(clf.cv_results_)
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [68]:
clf.best_score_

0.9800000000000001

one problem in GridSearchCV is competion cost (c [])

In [69]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(iris.data, iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [70]:
df = pd.DataFrame(rs.cv_results_)
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.98
1,10,rbf,0.98


chossing best model

In [71]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [76]:
model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [78]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
